# Deep Universal Regular Conditional Expectations:

---
This implements the universal deep neural model of $\mathcal{NN}_{1_{\mathbb{R}^n},\mathcal{D}}^{\sigma:\star}$ [Anastasis Kratsios](https://people.math.ethz.ch/~kratsioa/) - 2021.

---

## What does this code do?
1. Learn Heteroskedastic Non-Linear Regression Problem
     - $Y\sim f_{\text{unkown}}(x) + \epsilon$ where $f$ is an known function and $\epsilon\sim Laplace(0,\|x\|)$
2. Learn Random Bayesian Network's Law:
    - $Y = W_J Y^{J-1}, \qquad Y^{j}\triangleq \sigma\bullet A^{j}Y^{j-1} + b^{j}, \qquad Y^0\triangleq x$

3. In the above example if $A_j = M_j\odot \tilde{A_j}$ where $\tilde{A}_j$ is a deterministic matrix and $M_j$ is a "mask", that is, a random matrix with binary entries and $\odot$ is the Hadamard product then we recover the dropout framework.
4. Learn the probability distribution that the unique strong solution to the rough SDE with uniformly Lipschitz drivers driven by a factional Brownian motion with Hurst exponent $H \in [\frac1{2},1)$:
$$
X_t^x = x + \int_0^t \alpha(s,X_s^x)ds + \int_0^t \beta(s,X_s^x)dB_s^H
$$
belongs, at time $t=1$, to a ball about the initial point $x$ of random radius given by an independant exponential random-variable with shape parameter $\lambda=2$
5. Train a DNN to predict the returns of bitcoin with GD.  Since this has random initialization then each prediction of a given $x$ is stochastic...We learn the distribution of this conditional RV (conditioned on x in the input space).
$$
Y_x \triangleq \hat{f}_{\theta_{T}}(x), \qquad \theta_{(t+1)}\triangleq \theta_{(t)} + \lambda \sum_{x \in \mathbb{X}} \nabla_{\theta}\|\hat{f}_{\theta_t}(x) - f(x)\|, \qquad \theta_0 \sim N_d(0,1);
$$
$T\in \mathbb{N}$ is a fixed number of "SGD" iterations (typically identified by cross-validation on a single SGD trajectory for a single initialization) and where $\theta \in \mathbb{R}^{(d_{J}+1)+\sum_{j=0}^{J-1} (d_{j+1}d_j + 1)}$ and $d_j$ is the dimension of the "bias" vector $b_j$ defining each layer of the DNN with layer dimensions:
$$
\hat{f}_{\theta}(x)\triangleq A^{(J)}x^{(J)} + b^{(J)},\qquad x^{(j+1)}\sigma\bullet A^{j}x^{(j)} + b^{j},\qquad x^{(0)}\triangleq x
.
$$

In [1]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())

Using TensorFlow backend.


#### Mode:
Software/Hardware Testing or Real-Deal?

In [2]:
trial_run = True

### Simulation Method:

In [3]:
# # Random DNN
# f_unknown_mode = "Heteroskedastic_NonLinear_Regression"

# # Random DNN internal noise
# # f_unknown_mode = "DNN_with_Random_Weights"
Depth_Bayesian_DNN = 2
width = 20

# # Random Dropout applied to trained DNN
# f_unknown_mode = "DNN_with_Bayesian_Dropout"
Dropout_rate = 0.1

# # Rough SDE (time 1)
# f_unknown_mode = "Rough_SDE"

# GD with Randomized Input
f_unknown_mode = "GD_with_randomized_input"
GD_epochs = 2

## Problem Dimension

In [4]:
problem_dim = 2

## Note: *Why the procedure is so computationally efficient*?
---
 - The sample barycenters do not require us to solve for any new Wasserstein-1 Barycenters; which is much more computationally costly,
 - Our training procedure never back-propages through $\mathcal{W}_1$ since steps 2 and 3 are full-decoupled.  Therefore, training our deep classifier is (comparatively) cheap since it takes values in the standard $N$-simplex.

---

#### Rough SDE Meta-Parameters

In [5]:
# SDE with Rough Driver
N_Euler_Steps = 10**1
Hurst_Exponent = 0.01

def alpha(t,x):
    output_drift_update = t-x
    return output_drift_update

def beta(t,x):
    output_vol_update = (t+0.001)*np.diag(np.cos(x))
    return output_vol_update

#### Grid Hyperparameter(s)
- Ratio $\frac{\text{Testing Datasize}}{\text{Training Datasize}}$.
- Number of Training Points to Generate

In [6]:
train_test_ratio = .2
N_train_size = 10**1

Monte-Carlo Paramters

In [7]:
## Monte-Carlo
N_Monte_Carlo_Samples = 10**1

Initial radis of $\delta$-bounded random partition of $\mathcal{X}$!

In [8]:
# Hyper-parameters of Cover
delta = 0.01
Proportion_per_cluster = .01

# Run Main:

In [9]:
print("------------------------------")
print("Running script for main model!")
print("------------------------------")
# %run Universal_Measure_Valued_Networks_Backend.ipynb
exec(open('Universal_Measure_Valued_Networks_Backend.py').read())

print("------------------------------------")
print("Done: Running script for main model!")
print("------------------------------------")

100%|██████████| 2/2 [00:00<00:00, 6091.94it/s]

------------------------------
Running script for main model!
------------------------------
Deep Feature Builder - Ready
Deep Classifier - Ready
Deep Feature Builder - Ready
Deep Feature Builder - Ready
Deep Classifier - Ready
Training Classifer Portion of Type-A Model
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    3.6s finished


Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8764 - accuracy: 0.4000
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8714 - accuracy: 0.3000
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8664 - accuracy: 0.3000
Epoch 4/50
1/1 [==============================] - 0s 3ms/step - loss: 0.8614 - accuracy: 0.3000
Epoch 5/50
1/1 [==============================] - 0s 2ms/step - loss: 0.8565 - accuracy: 0.3000
Epoch 6/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8517 - accuracy: 0.3000
Epoch 7/50
1/1 [==============================] - 0s 4ms/step - loss: 0.8469 - accuracy: 0.3000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.8422 - accuracy: 0.3000
Epoch 9/50
1/1 [==============================] - 0s 1ms/step - loss: 0.8374 - accuracy: 0.3000
Epoch 10/50
1/1 [==============================] - 0s 2ms/step - loss: 0.8328 - accuracy: 0.3000
Epoch 11/50
1/1 [======================

100%|██████████| 2/2 [00:00<00:00, 547.31it/s]

Training Classifer Portion of Type Model: Done!
#--------------------#
 Get Training Error(s)
#--------------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
#----------------#
 Get Test Error(s)
#----------------#
#-------------------------#
 Get Training Error(s): END
#-------------------------#
------------------------------------
Done: Running script for main model!
------------------------------------


---
# Run: All Benchmarks

## 1) *Pointmass Benchmark(s)*
These benchmarks consist of subsets of $C(\mathbb{R}^d,\mathbb{R})$ which we lift to models in $C(\mathbb{R}^d,\cap_{1\leq q<\infty}\mathscr{P}_{q}(\mathbb{R}))$ via:
$$
\mathbb{R}^d \ni x \to f(x) \to \delta_{f(x)}\in \cap_{1\leq q<\infty}\mathcal{P}_{q}(\mathbb{R}).
$$

In [10]:
exec(open('CV_Grid.py').read())
# Notebook Mode:
# %run Evaluation.ipynb
# %run Benchmarks_Model_Builder_Pointmass_Based.ipynb
# Terminal Mode (Default):
exec(open('Evaluation.py').read())
exec(open('Benchmarks_Model_Builder_Pointmass_Based.py').read())

Deep Feature Builder - Ready
--------------
Training: ENET
--------------


100%|██████████| 2/2 [00:00<00:00, 810.10it/s]

#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
-----------------
Training: K-Ridge
-----------------
Fitting 2 folds for each of 2 candidates, totalling 4 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0354s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.3s finished
100%|██████████| 2/2 [00:00<00:00, 1167.19it/s]
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
--------------
Training: GBRF
--------------
Fitting 2 folds for each of 1 candidates, totalling 2 fits


...

Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 0.3882 - mse: 0.1630 - mae: 0.3882 - mape: 114.0914
Epoch 2/50
1/1 [==============================] - 0s 2ms/step - loss: 0.3882 - mse: 0.1630 - mae: 0.3882 - mape: 114.0825
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 0.3881 - mse: 0.1630 - mae: 0.3881 - mape: 114.0736
Epoch 4/50
1/1 [==============================] - 0s 2ms/step - loss: 0.3881 - mse: 0.1630 - mae: 0.3881 - mape: 114.0646
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: 0.3881 - mse: 0.1629 - mae: 0.3881 - mape: 114.0557
Epoch 6/50
1/1 [==============================] - 0s 2ms/step - loss: 0.3880 - mse: 0.1629 - mae: 0.3880 - mape: 114.0467
Epoch 7/50
1/1 [==============================] - 0s 8ms/step - loss: 0.3880 - mse: 0.1629 - mae: 0.3880 - mape: 114.0378
Epoch 8/50
1/1 [==============================] - 0s 2ms/step - loss: 0.3880 - mse: 0.1629 - mae: 0.3880 - mape: 114.0288
Epoch 9/50
1/1 [========

  0%|          | 0/10 [00:00<?, ?it/s]

#------------#
 Get Error(s) 
#------------#


100%|██████████| 2/2 [00:00<00:00, 655.67it/s]

#-----------------#
 Get Error(s): END 
#-----------------#
#------------#
 Get Error(s) 
#------------#
#-----------------#
 Get Error(s): END 
#-----------------#
-----------------------
Computing Error Metrics
-----------------------


# Summary of Point-Mass Regression Models

#### Training Model Facts

In [11]:
print(Summary_pred_Qual_models)
Summary_pred_Qual_models

                  ENET        kRidge          GBRF          ffNN
W1        2.615522e-04  6.434712e-04  5.522462e-03  1.537479e-01
Mean      1.333822e-02  1.940543e-02  7.273067e-02  3.638196e-01
Var       8.222246e-05  8.222246e-05  8.222246e-05  8.222246e-05
Skewness  5.514620e+05  5.514620e+05  5.514620e+05  5.514620e+05
Ex_Kur    4.426439e+08  4.426439e+08  4.426439e+08  4.426439e+08


,ENET,kRidge,GBRF,ffNN
W1,2.615522e-04,6.434712e-04,5.522462e-03,1.537479e-01
Mean,1.333822e-02,1.940543e-02,7.273067e-02,3.638196e-01
Var,8.222246e-05,8.222246e-05,8.222246e-05,8.222246e-05
Skewness,5.514620e+05,5.514620e+05,5.514620e+05,5.514620e+05
Ex_Kur,4.426439e+08,4.426439e+08,4.426439e+08,4.426439e+08


#### Testing Model Facts

In [12]:
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

                  ENET        kRidge          GBRF          ffNN
W1        4.868745e-03  4.937549e-03  2.571989e-03  1.627244e-01
Mean      5.508836e-02  5.449849e-02  3.564712e-02  3.867625e-01
Var       8.119958e-04  8.119958e-04  8.119958e-04  8.119958e-04
Skewness  1.540415e+05  1.540415e+05  1.540415e+05  1.540415e+05
Ex_Kur    3.827872e+07  3.827872e+07  3.827872e+07  3.827872e+07


,ENET,kRidge,GBRF,ffNN
W1,4.868745e-03,4.937549e-03,2.571989e-03,1.627244e-01
Mean,5.508836e-02,5.449849e-02,3.564712e-02,3.867625e-01
Var,8.119958e-04,8.119958e-04,8.119958e-04,8.119958e-04
Skewness,1.540415e+05,1.540415e+05,1.540415e+05,1.540415e+05
Ex_Kur,3.827872e+07,3.827872e+07,3.827872e+07,3.827872e+07


#### Model Complexitie(s)

In [13]:
print(Summary_Complexity_models)
Summary_Complexity_models

        N_Params_Trainable  N_Params    T_Time  T_Test/T_test-MC
ENET                     4         4  3.835630          0.011919
GBRF                  5000      5000  0.329652          0.036505
kRidge                  10        10  0.375161          0.032714
ffNN                    81        81  5.310286          8.588391


,N_Params_Trainable,N_Params,T_Time,T_Test/T_test-MC
ENET,4,4,3.835630,0.011919
GBRF,5000,5000,0.329652,0.036505
kRidge,10,10,0.375161,0.032714
ffNN,81,81,5.310286,8.588391


## 2) *Gaussian Benchmarks*

- Bencharm 1: [Gaussian Process Regressor](https://scikit-learn.org/stable/modules/gaussian_process.html)
- Benchmark 2: Deep Gaussian Networks:
These models train models which assume Gaussianity.  We may view these as models in $\mathcal{P}_2(\mathbb{R})$ via:
$$
\mathbb{R}^d \ni x \to (\hat{\mu}(x),\hat{\sigma}(x))\triangleq f(x) \in \mathbb{R}\times [0,\infty) \to \frac1{\hat{\sigma}(x)\sqrt{2\pi}}\exp\left(\frac{-(\cdot-\hat{\mu}(x))^2}{\hat{\sigma(x)}^2}\right) \in \mathcal{G}_1\subset \mathcal{P}_2(\mathbb{R});
$$
where $\mathcal{G}_1$ is the set of Gaussian measures on $\mathbb{R}$ equipped with the relative Wasserstein-1 topology.

Examples of this type of architecture are especially prevalent in uncertainty quantification; see ([Deep Ensembles](https://arxiv.org/abs/1612.01474)] or [NOMU: Neural Optimization-based Model Uncertainty](https://arxiv.org/abs/2102.13640).  Moreover, their universality in $C(\mathbb{R}^d,\mathcal{G}_2)$ is known, and has been shown in [Corollary 4.7](https://arxiv.org/abs/2101.05390).

In [14]:
# %run Benchmarks_Model_Builder_Mean_Var.ipynb
exec(open('Benchmarks_Model_Builder_Mean_Var.py').read())

Deep Feature Builder - Ready
Deep Feature Builder - Ready
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0671s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
100%|██████████| 10/10 [00:00<00:00, 65.97it/s]

Infering Parameters for Deep Gaussian Network to train on!
Done Getting Parameters for Deep Gaussian Network!
Training Deep Gaussian Network!
Fitting 2 folds for each of 1 candidates, totalling 2 fits



[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:    1.1s finished


Epoch 1/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6603 - mse: 0.5577 - mae: 0.6603 - mape: 2662.9827
Epoch 2/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6603 - mse: 0.5577 - mae: 0.6603 - mape: 2662.9224
Epoch 3/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6603 - mse: 0.5577 - mae: 0.6603 - mape: 2662.8616
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6603 - mse: 0.5577 - mae: 0.6603 - mape: 2662.8013
Epoch 5/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6603 - mse: 0.5577 - mae: 0.6603 - mape: 2662.7407
Epoch 6/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6603 - mse: 0.5576 - mae: 0.6603 - mape: 2662.6802
Epoch 7/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6603 - mse: 0.5576 - mae: 0.6603 - mape: 2662.6191
Epoch 8/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6603 - mse: 0.5576 - mae: 0.6603 - mape: 2662.5588
Epoch 9/50
1/1 [

  0%|          | 0/10 [00:00<?, ?it/s]

Training Deep Gaussian Network!: END
#---------------------------------------#
 Get Training Errors for: Gaussian Models
#---------------------------------------#


100%|██████████| 2/2 [00:00<00:00, 996.63it/s]

#-------------------------#
 Get Training Error(s): END
#-------------------------#
#---------------------------------------#
 Get Testing Errors for: Gaussian Models
#---------------------------------------#
#------------------------#
 Get Testing Error(s): END
#------------------------#
-------------------------------------------------
Updating Performance Metrics Dataframe and Saved!
-------------------------------------------------
------------------------------------------------
Updated Performance Metrics Dataframe and Saved!
------------------------------------------------
--------------------------------------------
Computing and Updating Complexity Metrics...
--------------------------------------------
-----------------------------------------------
Updated Complexity Metrics Dataframe and Saved!
-----------------------------------------------


In [15]:
print("Prediction Quality (Updated)")
print(Summary_pred_Qual_models_test)
Summary_pred_Qual_models_test

Prediction Quality (Updated)
                  ENET        kRidge          GBRF          ffNN  \
W1        4.868745e-03  4.937549e-03  2.571989e-03  1.627244e-01   
Mean      5.508836e-02  5.449849e-02  3.564712e-02  3.867625e-01   
Var       8.119958e-04  8.119958e-04  8.119958e-04  8.119958e-04   
Skewness  1.540415e+05  1.540415e+05  1.540415e+05  1.540415e+05   
Ex_Kur    3.827872e+07  3.827872e+07  3.827872e+07  3.827872e+07   

                   GPR           DGN  
W1        5.951454e-03  1.276742e+00  
Mean      6.581447e-02  2.974808e-01  
Var       6.573225e-02  1.024494e+00  
Skewness  0.000000e+00  0.000000e+00  
Ex_Kur    3.033505e+13  3.033505e+13  


,ENET,kRidge,GBRF,ffNN,GPR,DGN
W1,4.868745e-03,4.937549e-03,2.571989e-03,1.627244e-01,5.951454e-03,1.276742e+00
Mean,5.508836e-02,5.449849e-02,3.564712e-02,3.867625e-01,6.581447e-02,2.974808e-01
Var,8.119958e-04,8.119958e-04,8.119958e-04,8.119958e-04,6.573225e-02,1.024494e+00
Skewness,1.540415e+05,1.540415e+05,1.540415e+05,1.540415e+05,0.000000e+00,0.000000e+00
Ex_Kur,3.827872e+07,3.827872e+07,3.827872e+07,3.827872e+07,3.033505e+13,3.033505e+13


In [16]:
print("Model Complexities Quality (Updated)")
print(Summary_Complexity_models)
Summary_Complexity_models

Model Complexities Quality (Updated)
        N_Params_Trainable  N_Params        T_Time  T_Test/T_test-MC
ENET                   4.0       4.0  3.835630e+00          0.011919
GBRF                5000.0    5000.0  3.296518e-01          0.036505
kRidge                10.0      10.0  3.751614e-01          0.032714
ffNN                  81.0      81.0  5.310286e+00          8.588391
GPR                    0.0       0.0  3.968196e-01          0.036591
DGN                   81.0      81.0  1.619614e+09          9.390068


,N_Params_Trainable,N_Params,T_Time,T_Test/T_test-MC
ENET,4.0,4.0,3.835630e+00,0.011919
GBRF,5000.0,5000.0,3.296518e-01,0.036505
kRidge,10.0,10.0,3.751614e-01,0.032714
ffNN,81.0,81.0,5.310286e+00,8.588391
GPR,0.0,0.0,3.968196e-01,0.036591
DGN,81.0,81.0,1.619614e+09,9.390068


# 3) The natural Universal Benchmark: [Bishop's Mixture Density Network](https://publications.aston.ac.uk/id/eprint/373/1/NCRG_94_004.pdf)

This implementation is as follows:
- For every $x$ in the trainingdata-set we fit a GMM $\hat{\nu}_x$, using the [Expectation-Maximization (EM) algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm), with the same number of centers as the deep neural model in $\mathcal{NN}_{1_{\mathbb{R}^d},\mathcal{D}}^{\sigma:\star}$ which we are evaluating.  
- A Mixture density network is then trained to predict the infered parameters; given any $x \in \mathbb{R}^d$.

In [ ]:
# %run Mixture_Density_Network.ipynb
exec(open('Mixture_Density_Network.py').read())

  0%|          | 0/10 [00:00<?, ?it/s]

Preparing Training Outputs for MDNs using EM-Algorithm


100%|██████████| 10/10 [00:00<00:00, 168.51it/s]


Prepared Training Outputs for MDNs using EM-Algorithm!
Deep Feature Builder - Ready
(0)
Training Mixture Density Network (MDN): Means: Start!


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 2 folds for each of 1 candidates, totalling 2 fits


## Get Final Outputs
Now we piece together all the numerical experiments and report a nice summary.

In [ ]:
# %run WrapUp_Summarizer.ipynb
exec(open('WrapUp_Summarizer.py').read())

# Result(s)

## Model Complexities

In [ ]:
Summary_Complexity_models

## Prediction Quality

#### Training

In [ ]:
PredictivePerformance_Metrics_Train

#### Test

In [ ]:
PredictivePerformance_Metrics_Test

# For Terminal Runner(s):

In [ ]:
# For Terminal Running
print(" ")
print(" ")
print(" ")
print("====================================")
print("Model Complexity Predictive Quality:")
print("====================================")
print(" ")
print(" ")
print(" ")
print(Summary_Complexity_models)
print(" ")
print(" ")
print(" ")
print("============================")
print("Training Predictive Quality:")
print("============================")
print(PredictivePerformance_Metrics_Train)
print(" ")
print(" ")
print(" ")
print("===========================")
print("Testing Predictive Quality:")
print("===========================")
print(PredictivePerformance_Metrics_Test)
print("================================")
print(" ")
print(" ")
print(" ")
print("Kernel_Used_in_GPR: "+str(GPR_trash.kernel))
print("🙃🙃 Have a wonderful day! 🙃🙃")

---
# Fin
---

---